---

# Chapter 4: Probability & Information Theory

---

Dealing with **uncertainty** is fundamental to AI. Probability provides the language to describe uncertainty, and **information theory** provides measures of it.

> **Key Insight**: The real world is noisy and uncertain. Probabilistic models embrace this reality rather than fighting it.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

## 4.1 Entropy and KL Divergence

### Theory

**Entropy** $H(P)$ measures the uncertainty or randomness in a distribution:

$$H(P) = -\sum_x P(x) \log P(x)$$

Properties:
- Maximum for uniform distribution
- Zero for deterministic distribution (one outcome has probability 1)

**KL Divergence** $D_{KL}(P||Q)$ measures how different distribution $Q$ is from $P$:

$$D_{KL}(P||Q) = \sum_x P(x) \log \frac{P(x)}{Q(x)}$$

Properties:
- Always non-negative ($\geq 0$)
- Zero if and only if $P = Q$
- **Not symmetric**: $D_{KL}(P||Q) \neq D_{KL}(Q||P)$

### 📐 Mathematical Example

**Distributions**:
- $P = [0.25, 0.75]$ (biased coin)
- $Q = [0.5, 0.5]$ (fair coin)

**Entropy of P**:
$$H(P) = -0.25 \log(0.25) - 0.75 \log(0.75) = 0.562 \text{ bits}$$

**KL Divergence**:
$$D_{KL}(P||Q) = 0.25 \log\frac{0.25}{0.5} + 0.75 \log\frac{0.75}{0.5} = 0.0589 \text{ bits}$$

In [ ]:
# ============================================
# 4.1 ENTROPY AND KL DIVERGENCE
# ============================================

def entropy(p: np.ndarray) -> float:
    """Compute entropy in bits."""
    p = np.array(p)
    p = p[p > 0]  # Avoid log(0)
    return -np.sum(p * np.log2(p))


def kl_divergence(p: np.ndarray, q: np.ndarray) -> float:
    """Compute KL divergence D_KL(P||Q) in bits."""
    p, q = np.array(p), np.array(q)
    # Only consider where p > 0
    mask = p > 0
    return np.sum(p[mask] * np.log2(p[mask] / q[mask]))


# Example distributions
P = np.array([0.25, 0.75])  # Biased coin
Q = np.array([0.5, 0.5])   # Fair coin
R = np.array([0.1, 0.9])   # Very biased

print("="*60)
print("ENTROPY AND KL DIVERGENCE")
print("="*60)

print(f"\n📊 Distributions:")
print(f"  P (biased):      {P}")
print(f"  Q (fair coin):   {Q}")
print(f"  R (very biased): {R}")

print(f"\n📐 Entropy (uncertainty):")
print(f"  H(P) = {entropy(P):.4f} bits")
print(f"  H(Q) = {entropy(Q):.4f} bits (maximum for 2 outcomes)")
print(f"  H(R) = {entropy(R):.4f} bits (low uncertainty)")

print(f"\n📐 KL Divergence (difference from Q):")
print(f"  D_KL(P||Q) = {kl_divergence(P, Q):.4f} bits")
print(f"  D_KL(R||Q) = {kl_divergence(R, Q):.4f} bits (R is more different from Q)")
print(f"  D_KL(Q||Q) = {kl_divergence(Q, Q):.4f} bits (same distribution = 0)")

In [ ]:
# Visualization: Entropy vs probability
p_range = np.linspace(0.001, 0.999, 100)
entropy_values = [entropy([p, 1-p]) for p in p_range]

plt.figure(figsize=(10, 5))
plt.plot(p_range, entropy_values, 'b-', linewidth=2)
plt.axvline(x=0.5, color='r', linestyle='--', label='Maximum entropy (p=0.5)')
plt.scatter([0.25, 0.5, 0.1], [entropy([0.25, 0.75]), entropy([0.5, 0.5]), entropy([0.1, 0.9])],
            c=['green', 'red', 'orange'], s=100, zorder=5)
plt.annotate('P', (0.25, entropy([0.25, 0.75])+0.05), fontsize=12, fontweight='bold')
plt.annotate('Q', (0.5, entropy([0.5, 0.5])+0.05), fontsize=12, fontweight='bold')
plt.annotate('R', (0.1, entropy([0.1, 0.9])+0.05), fontsize=12, fontweight='bold')
plt.xlabel('Probability p (for outcome 1)', fontsize=12)
plt.ylabel('Entropy H(p) [bits]', fontsize=12)
plt.title('Binary Entropy Function', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("💡 Entropy is maximum when outcomes are equally likely (maximum uncertainty).")

## 4.2 Variational Autoencoders (VAE)

### Theory

**VAEs** are generative models that learn to map data to a latent space that follows a standard Gaussian $N(0, I)$.

The loss function is the **Evidence Lower Bound (ELBO)**:

$$\mathcal{L} = \underbrace{-\mathbb{E}_{q(z|x)}[\log p(x|z)]}_{\text{Reconstruction Loss}} + \underbrace{D_{KL}(q(z|x) || p(z))}_{\text{KL Regularization}}$$

### KL Between Two Gaussians (Closed Form)

For $q(z|x) = N(\mu, \sigma^2)$ and $p(z) = N(0, 1)$:

$$D_{KL}(q||p) = -\frac{1}{2} \sum_{j=1}^{d} (1 + \log(\sigma_j^2) - \mu_j^2 - \sigma_j^2)$$

### 📐 Mathematical Example

**Encoder output**: $\mu = 0.5$, $\sigma = 0.8$

$$D_{KL} = -\frac{1}{2}(1 + \log(0.64) - 0.25 - 0.64) = -\frac{1}{2}(1 - 0.446 - 0.25 - 0.64) = 0.168$$

In [ ]:
# ============================================
# 4.2 VAE KL DIVERGENCE COMPUTATION
# ============================================

def kl_divergence_gaussian(mu: np.ndarray, log_var: np.ndarray) -> float:
    """
    Compute KL divergence between N(mu, exp(log_var)) and N(0, 1).
    This is the closed-form solution used in VAEs.
    
    Args:
        mu: Mean of the encoder distribution
        log_var: Log variance of the encoder distribution
    
    Returns:
        KL divergence value
    """
    return -0.5 * np.sum(1 + log_var - mu**2 - np.exp(log_var))


def reparameterize(mu: np.ndarray, log_var: np.ndarray) -> np.ndarray:
    """
    Reparameterization trick: z = mu + sigma * epsilon
    where epsilon ~ N(0, 1)
    """
    std = np.exp(0.5 * log_var)
    eps = np.random.randn(*mu.shape)
    return mu + std * eps


# Example: encoder outputs
mu = np.array([0.5, -0.3, 0.8])
sigma = np.array([0.8, 0.5, 1.2])
log_var = 2 * np.log(sigma)  # log(sigma^2)

print("="*60)
print("VAE KL DIVERGENCE (CLOSED FORM)")
print("="*60)
print(f"Encoder output:")
print(f"  μ = {mu}")
print(f"  σ = {sigma}")
print(f"  log(σ²) = {log_var}")

kl = kl_divergence_gaussian(mu, log_var)
print(f"\nD_KL(q(z|x) || p(z)) = {kl:.4f}")

# Show reparameterization trick
print(f"\n🎲 Reparameterization samples:")
for i in range(3):
    z = reparameterize(mu, log_var)
    print(f"  z_{i+1} = {z}")

print("\n💡 The reparameterization trick allows gradients to flow through sampling!")

In [ ]:
# Visualization: Effect of KL divergence on latent space
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Different encoder distributions
configs = [
    (0, 1, 'Standard Normal N(0,1)'),
    (0.5, 0.8, 'Shifted: N(0.5, 0.64)'),
    (2, 0.3, 'Far from prior: N(2, 0.09)'),
]

x = np.linspace(-4, 5, 200)

for ax, (mu, sigma, title) in zip(axes, configs):
    # Prior p(z)
    prior = stats.norm.pdf(x, 0, 1)
    # Encoder q(z|x)
    encoder = stats.norm.pdf(x, mu, sigma)
    
    ax.fill_between(x, prior, alpha=0.3, label='Prior p(z) = N(0,1)')
    ax.fill_between(x, encoder, alpha=0.3, label=f'Encoder q(z|x)')
    ax.plot(x, prior, 'b-', linewidth=2)
    ax.plot(x, encoder, 'r-', linewidth=2)
    
    # Compute KL
    log_var = 2 * np.log(sigma)
    kl = kl_divergence_gaussian(np.array([mu]), np.array([log_var]))
    
    ax.set_title(f'{title}\nD_KL = {kl:.3f}', fontweight='bold')
    ax.set_xlabel('z')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

plt.suptitle('KL Divergence in VAE: Encoder vs Prior', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("📈 Higher KL = encoder distribution is far from the prior.")
print("📉 VAE training minimizes KL to keep latent space regular.")

## 4.3 t-SNE Algorithm

### Theory

**t-SNE** (t-Distributed Stochastic Neighbor Embedding) visualizes high-dimensional data in 2D/3D.

**Key idea**: Convert distances to probabilities, then minimize the difference between high-dim and low-dim probability distributions.

**High-dimensional space**: Gaussian similarities
$$p_{j|i} = \frac{\exp(-\|x_i - x_j\|^2 / 2\sigma_i^2)}{\sum_{k \neq i} \exp(-\|x_i - x_k\|^2 / 2\sigma_i^2)}$$

**Low-dimensional space**: t-distribution (heavier tails to prevent crowding)
$$q_{ij} = \frac{(1 + \|y_i - y_j\|^2)^{-1}}{\sum_{k \neq l} (1 + \|y_k - y_l\|^2)^{-1}}$$

**Loss**: KL divergence between P and Q
$$C = \sum_i D_{KL}(P_i || Q_i) = \sum_i \sum_j p_{ij} \log \frac{p_{ij}}{q_{ij}}$$

In [ ]:
# ============================================
# 4.3 t-SNE SIMPLIFIED DEMO
# ============================================

from sklearn.manifold import TSNE
from sklearn.datasets import make_blobs

# Generate high-dimensional clustered data
np.random.seed(42)
n_samples = 300
n_features = 50  # High dimensional
n_clusters = 4

X, y = make_blobs(n_samples=n_samples, n_features=n_features, 
                  centers=n_clusters, cluster_std=2.0, random_state=42)

print("="*60)
print("t-SNE DIMENSIONALITY REDUCTION")
print("="*60)
print(f"Original data: {X.shape[0]} samples × {X.shape[1]} dimensions")

# Apply t-SNE
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X)

print(f"t-SNE output: {X_tsne.shape[0]} samples × {X_tsne.shape[1]} dimensions")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Original (first 2 dimensions only)
scatter1 = axes[0].scatter(X[:, 0], X[:, 1], c=y, cmap='tab10', alpha=0.7)
axes[0].set_title('Original Data (First 2 of 50 dims)\nClusters not clearly separated', fontweight='bold')
axes[0].set_xlabel('Dimension 1')
axes[0].set_ylabel('Dimension 2')

# t-SNE
scatter2 = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='tab10', alpha=0.7)
axes[1].set_title('t-SNE Projection (2D)\nClusters clearly separated!', fontweight='bold')
axes[1].set_xlabel('t-SNE Dimension 1')
axes[1].set_ylabel('t-SNE Dimension 2')

plt.tight_layout()
plt.show()

print("\n💡 t-SNE reveals cluster structure hidden in high dimensions!")
print("   It uses KL divergence to preserve local neighborhood structure.")

---

# Chapter 5: Advanced Integration Methods

---

Many ML problems require computing integrals that have no closed-form solution. **Monte Carlo methods** provide numerical approximations.

## 5.1 Monte Carlo Integration and Importance Sampling

### Theory

**Monte Carlo Integration** approximates expectations:

$$\mathbb{E}_{x \sim p}[f(x)] = \int f(x) p(x) dx \approx \frac{1}{N} \sum_{i=1}^N f(x_i), \quad x_i \sim p$$

**Problem**: What if $p(x)$ is hard to sample from?

**Solution**: **Importance Sampling** - sample from an easier distribution $q(x)$:

$$\mathbb{E}_p[f(x)] = \mathbb{E}_q\left[f(x) \frac{p(x)}{q(x)}\right] \approx \frac{1}{N} \sum_{i=1}^N f(x_i) \frac{p(x_i)}{q(x_i)}, \quad x_i \sim q$$

The ratio $w_i = p(x_i)/q(x_i)$ is called the **importance weight**.

### 📐 Mathematical Example

Estimate $I = \int_0^1 e^x dx$ (true value: $e - 1 \approx 1.718$)

Using uniform proposal $q(x) = 1$ on $[0, 1]$:

$$\hat{I} = \frac{1}{N} \sum_{i=1}^N e^{x_i}$$

In [ ]:
# ============================================
# 5.1 MONTE CARLO AND IMPORTANCE SAMPLING
# ============================================

def monte_carlo_estimate(f, n_samples=10000):
    """Simple Monte Carlo: sample from Uniform[0,1]."""
    x = np.random.uniform(0, 1, n_samples)
    return np.mean(f(x))


def importance_sampling_estimate(f, p, q_sample, q_pdf, n_samples=10000):
    """
    Importance sampling estimate.
    
    Args:
        f: Function to integrate
        p: Target probability density
        q_sample: Function to sample from proposal
        q_pdf: Proposal probability density
    """
    x = q_sample(n_samples)
    weights = p(x) / q_pdf(x)
    return np.mean(f(x) * weights)


# Example: Integrate e^x from 0 to 1
f = lambda x: np.exp(x)
true_value = np.e - 1  # ≈ 1.718

print("="*60)
print("MONTE CARLO INTEGRATION")
print("="*60)
print(f"Integral: ∫₀¹ eˣ dx")
print(f"True value: {true_value:.6f}")

# Simple Monte Carlo
estimates = [monte_carlo_estimate(f, n) for n in [100, 1000, 10000, 100000]]
print(f"\n📊 Simple Monte Carlo estimates:")
for n, est in zip([100, 1000, 10000, 100000], estimates):
    print(f"  N={n:6d}: {est:.6f} (error: {abs(est - true_value):.6f})")

print("\n✅ Error decreases as √N (law of large numbers)")

In [ ]:
# Visualization: Convergence of Monte Carlo
np.random.seed(42)
n_max = 5000
samples = np.random.uniform(0, 1, n_max)
cumulative_mean = np.cumsum(np.exp(samples)) / (np.arange(1, n_max + 1))

plt.figure(figsize=(10, 5))
plt.plot(cumulative_mean, 'b-', alpha=0.7, linewidth=1)
plt.axhline(y=true_value, color='r', linestyle='--', linewidth=2, label=f'True value = {true_value:.4f}')
plt.fill_between(range(n_max), true_value - 0.1, true_value + 0.1, alpha=0.2, color='red')
plt.xlabel('Number of samples', fontsize=12)
plt.ylabel('Estimate', fontsize=12)
plt.title('Monte Carlo Convergence: ∫₀¹ eˣ dx', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, n_max)
plt.show()

print("💡 The estimate fluctuates but converges to the true value.")

## 5.2 Normalizing Flows

### Theory

**Normalizing Flows** transform a simple distribution (e.g., Gaussian) into a complex one through a series of invertible transformations.

Given $z \sim p_z(z)$ and an invertible function $x = f(z)$, the density of $x$ is:

$$p_x(x) = p_z(f^{-1}(x)) \left| \det \frac{\partial f^{-1}}{\partial x} \right| = p_z(z) \left| \det \frac{\partial f}{\partial z} \right|^{-1}$$

The key is choosing $f$ such that the **Jacobian determinant** is easy to compute.

### 📐 Mathematical Example: Affine Flow

Simplest flow: $x = az + b$ (scale and shift)

Jacobian determinant: $|a|$

$$p_x(x) = p_z\left(\frac{x-b}{a}\right) \cdot \frac{1}{|a|}$$

In [ ]:
# ============================================
# 5.2 NORMALIZING FLOWS DEMONSTRATION
# ============================================

class AffineFlow:
    """Simple affine flow: x = a*z + b"""
    
    def __init__(self, a: float, b: float):
        self.a = a
        self.b = b
    
    def forward(self, z: np.ndarray) -> np.ndarray:
        return self.a * z + self.b
    
    def inverse(self, x: np.ndarray) -> np.ndarray:
        return (x - self.b) / self.a
    
    def log_det_jacobian(self) -> float:
        return np.log(np.abs(self.a))


# Start with standard normal
np.random.seed(42)
z = np.random.randn(5000)

# Apply flow: x = 2z + 3 (scale by 2, shift by 3)
flow = AffineFlow(a=2.0, b=3.0)
x = flow.forward(z)

print("="*60)
print("NORMALIZING FLOWS")
print("="*60)
print(f"Base distribution: z ~ N(0, 1)")
print(f"Flow: x = 2z + 3")
print(f"Resulting distribution: x ~ N(3, 4)")
print(f"\nlog|det(J)| = log|2| = {flow.log_det_jacobian():.4f}")

# Verification
print(f"\n📊 Empirical statistics:")
print(f"  z: mean={np.mean(z):.3f}, std={np.std(z):.3f}")
print(f"  x: mean={np.mean(x):.3f}, std={np.std(x):.3f}")

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Base distribution
axes[0].hist(z, bins=50, density=True, alpha=0.7, color='blue')
x_plot = np.linspace(-4, 4, 100)
axes[0].plot(x_plot, stats.norm.pdf(x_plot), 'k-', linewidth=2)
axes[0].set_title('Base: z ~ N(0, 1)', fontweight='bold')
axes[0].set_xlabel('z')

# Flow transformation
axes[1].annotate('', xy=(0.9, 0.5), xytext=(0.1, 0.5),
                 arrowprops=dict(arrowstyle='->', lw=3, color='green'),
                 xycoords='axes fraction')
axes[1].text(0.5, 0.6, 'x = 2z + 3', fontsize=16, ha='center', transform=axes[1].transAxes, fontweight='bold')
axes[1].text(0.5, 0.4, 'Invertible!', fontsize=12, ha='center', transform=axes[1].transAxes)
axes[1].axis('off')
axes[1].set_title('Affine Flow', fontweight='bold')

# Transformed distribution
axes[2].hist(x, bins=50, density=True, alpha=0.7, color='orange')
x_plot = np.linspace(-3, 9, 100)
axes[2].plot(x_plot, stats.norm.pdf(x_plot, loc=3, scale=2), 'k-', linewidth=2)
axes[2].set_title('Result: x ~ N(3, 4)', fontweight='bold')
axes[2].set_xlabel('x')

plt.suptitle('Normalizing Flow: Transform Simple → Complex', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 By stacking many invertible layers, flows can model very complex distributions!")

---

# Chapter 6: Network Analysis

---

## 6.1 Supervised Random Walks (Facebook Link Prediction)

### Theory

**Link Prediction**: Given a social network, predict which new edges (friendships) will form.

**Supervised Random Walks** (Backstrom & Leskovec, 2011) learn edge weights based on features to bias a random walk toward future friends.

**Key idea**: Learn a function $\text{strength}(u, v) = f_w(\text{features}(u, v))$ that weights edges.

The optimization objective:

$$\min_w \|w\|^2 + \lambda \sum_{(d, l) \in D} h(p_l - p_d)$$

Where:
- $d$: A node that becomes a friend (destination)
- $l$: A node that does not (non-link)
- $p_d, p_l$: Random walk probabilities of reaching these nodes
- $h$: Hinge-like loss

In [ ]:
# ============================================
# 6.1 LINK PREDICTION DEMO
# ============================================

def compute_common_neighbors(adj: np.ndarray) -> np.ndarray:
    """Compute common neighbors score for all node pairs."""
    # CN(u,v) = |N(u) ∩ N(v)| = (A @ A)[u,v]
    return adj @ adj


def compute_adamic_adar(adj: np.ndarray) -> np.ndarray:
    """Compute Adamic-Adar score for all node pairs."""
    n = adj.shape[0]
    degrees = adj.sum(axis=1)
    scores = np.zeros((n, n))
    
    for u in range(n):
        for v in range(n):
            if u != v:
                # Find common neighbors
                common = np.where((adj[u] == 1) & (adj[v] == 1))[0]
                # Sum 1/log(degree) for each common neighbor
                for w in common:
                    if degrees[w] > 1:
                        scores[u, v] += 1 / np.log(degrees[w])
    
    return scores


# Create a sample social network
# 6 users: edges represent friendships
adj = np.array([
    [0, 1, 1, 0, 0, 0],  # User 0: friends with 1, 2
    [1, 0, 1, 1, 0, 0],  # User 1: friends with 0, 2, 3
    [1, 1, 0, 1, 1, 0],  # User 2: friends with 0, 1, 3, 4
    [0, 1, 1, 0, 1, 1],  # User 3: friends with 1, 2, 4, 5
    [0, 0, 1, 1, 0, 1],  # User 4: friends with 2, 3, 5
    [0, 0, 0, 1, 1, 0],  # User 5: friends with 3, 4
])

print("="*60)
print("LINK PREDICTION IN SOCIAL NETWORKS")
print("="*60)

# Compute scores
cn_scores = compute_common_neighbors(adj)
aa_scores = compute_adamic_adar(adj)

# Find potential links (pairs that aren't already connected)
print("\n📊 Potential new friendships (ranked by Common Neighbors):")
potential_links = []
for i in range(6):
    for j in range(i+1, 6):
        if adj[i, j] == 0:  # Not already friends
            potential_links.append((i, j, cn_scores[i, j], aa_scores[i, j]))

# Sort by common neighbors
potential_links.sort(key=lambda x: x[2], reverse=True)

print(f"{'Pair':<10} {'Common Neighbors':<18} {'Adamic-Adar':<12}")
print("-" * 40)
for i, j, cn, aa in potential_links[:5]:
    print(f"({i}, {j})     {int(cn):<18} {aa:.3f}")

In [ ]:
# Visualization of the network
import matplotlib.patches as mpatches

# Node positions (manually arranged for visualization)
pos = {
    0: (0, 1),
    1: (1, 2),
    2: (2, 1),
    3: (3, 2),
    4: (4, 1),
    5: (4, 2.5)
}

plt.figure(figsize=(10, 6))

# Draw existing edges
for i in range(6):
    for j in range(i+1, 6):
        if adj[i, j] == 1:
            plt.plot([pos[i][0], pos[j][0]], [pos[i][1], pos[j][1]], 
                     'b-', linewidth=2, alpha=0.6)

# Draw predicted edge (highest score)
best_i, best_j = potential_links[0][0], potential_links[0][1]
plt.plot([pos[best_i][0], pos[best_j][0]], [pos[best_i][1], pos[best_j][1]], 
         'g--', linewidth=3, label='Predicted friendship')

# Draw nodes
for node, (x, y) in pos.items():
    plt.scatter(x, y, c='steelblue', s=800, zorder=5, edgecolors='black', linewidths=2)
    plt.annotate(f'User {node}', (x, y), fontsize=10, ha='center', va='center', color='white', fontweight='bold')

plt.title('Social Network with Predicted Link', fontsize=14, fontweight='bold')
plt.legend(loc='upper left')
plt.axis('off')
plt.tight_layout()
plt.show()

print(f"\n🎯 Most likely new friendship: Users {best_i} and {best_j}")
print(f"   (They have {int(potential_links[0][2])} common friends!)")

---

# Chapter 7: Bayesian Optimization

---

When the objective function is expensive to evaluate (hyperparameter tuning, drug discovery), gradient-based methods are impractical. **Bayesian Optimization** uses a probabilistic model to guide the search.

## 7.1 Gaussian Processes

### Theory

A **Gaussian Process (GP)** is a distribution over functions:

$$f(x) \sim \mathcal{GP}(m(x), k(x, x'))$$

Where:
- $m(x)$: Mean function
- $k(x, x')$: Kernel (covariance) function

**Key property**: GP gives both a prediction AND uncertainty!

### Squared Exponential (RBF) Kernel

$$k(x, x') = \sigma^2 \exp\left(-\frac{\|x - x'\|^2}{2\ell^2}\right)$$

Parameters:
- $\sigma^2$: Signal variance (amplitude)
- $\ell$: Length scale (how quickly function varies)

In [ ]:
# ============================================
# 7.1 GAUSSIAN PROCESS REGRESSION
# ============================================

def rbf_kernel(X1: np.ndarray, X2: np.ndarray, 
               length_scale: float = 1.0, variance: float = 1.0) -> np.ndarray:
    """
    Compute RBF (squared exponential) kernel matrix.
    """
    X1 = X1.reshape(-1, 1) if X1.ndim == 1 else X1
    X2 = X2.reshape(-1, 1) if X2.ndim == 1 else X2
    
    # Compute squared Euclidean distances
    sqdist = np.sum(X1**2, axis=1, keepdims=True) + \
             np.sum(X2**2, axis=1) - 2 * X1 @ X2.T
    
    return variance * np.exp(-0.5 * sqdist / (length_scale**2))


def gp_predict(X_train: np.ndarray, y_train: np.ndarray, 
               X_test: np.ndarray, length_scale: float = 1.0,
               noise: float = 1e-6):
    """
    GP prediction with posterior mean and variance.
    """
    K = rbf_kernel(X_train, X_train, length_scale) + noise * np.eye(len(X_train))
    K_star = rbf_kernel(X_train, X_test, length_scale)
    K_star_star = rbf_kernel(X_test, X_test, length_scale)
    
    # Compute posterior
    K_inv = np.linalg.inv(K)
    mu = K_star.T @ K_inv @ y_train
    cov = K_star_star - K_star.T @ K_inv @ K_star
    
    return mu, np.sqrt(np.diag(cov))


# Generate training data
np.random.seed(42)
X_train = np.array([1, 3, 5, 6, 7])
y_train = np.sin(X_train) + 0.1 * np.random.randn(len(X_train))

# Test points
X_test = np.linspace(0, 10, 100)

# GP prediction
mu, std = gp_predict(X_train, y_train, X_test, length_scale=1.0)

print("="*60)
print("GAUSSIAN PROCESS REGRESSION")
print("="*60)
print(f"Training points: {len(X_train)}")
print(f"Test points: {len(X_test)}")
print(f"\nGP provides:")
print(f"  - Mean prediction μ(x)")
print(f"  - Uncertainty σ(x) at each point!")

In [ ]:
# Visualization
plt.figure(figsize=(12, 5))

# True function
plt.plot(X_test, np.sin(X_test), 'k--', label='True function: sin(x)', linewidth=2)

# GP prediction
plt.plot(X_test, mu, 'b-', label='GP Mean prediction', linewidth=2)
plt.fill_between(X_test, mu - 2*std, mu + 2*std, alpha=0.3, color='blue', label='95% confidence')

# Training points
plt.scatter(X_train, y_train, c='red', s=100, zorder=5, label='Training data', edgecolors='black')

plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.title('Gaussian Process: Prediction with Uncertainty', fontsize=14, fontweight='bold')
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.show()

print("💡 Notice: Uncertainty is LOW near training points, HIGH far from them!")

## 7.2 Acquisition Functions

### Theory

**Acquisition functions** decide where to sample next by balancing:
- **Exploitation**: Sample where we expect good values
- **Exploration**: Sample where we're uncertain

### Expected Improvement (EI)

$$\alpha_{EI}(x) = (\mu(x) - f^* - \xi) \Phi(Z) + \sigma(x) \phi(Z)$$

Where:
- $Z = \frac{\mu(x) - f^* - \xi}{\sigma(x)}$
- $f^*$: Best value observed so far
- $\xi$: Exploration parameter
- $\Phi$, $\phi$: CDF and PDF of standard normal

In [ ]:
# ============================================
# 7.2 BAYESIAN OPTIMIZATION WITH EI
# ============================================

def expected_improvement(mu: np.ndarray, std: np.ndarray, 
                         f_best: float, xi: float = 0.01) -> np.ndarray:
    """
    Compute Expected Improvement acquisition function.
    """
    std = np.maximum(std, 1e-9)  # Avoid division by zero
    Z = (mu - f_best - xi) / std
    ei = (mu - f_best - xi) * stats.norm.cdf(Z) + std * stats.norm.pdf(Z)
    return ei


# Objective function to optimize (expensive black-box function)
def objective(x):
    return -((x - 2)**2 * np.sin(3*x))


# Initial samples
X_train = np.array([0.5, 2.0, 4.5])
y_train = objective(X_train)
f_best = np.max(y_train)

# Test points
X_test = np.linspace(0, 5, 200)

# GP prediction
mu, std = gp_predict(X_train, y_train, X_test, length_scale=0.5)

# Expected Improvement
ei = expected_improvement(mu, std, f_best)

print("="*60)
print("BAYESIAN OPTIMIZATION")
print("="*60)
print(f"Current best: {f_best:.4f} at x = {X_train[np.argmax(y_train)]:.2f}")
print(f"Next point to sample: x = {X_test[np.argmax(ei)]:.4f}")
print(f"Expected Improvement at that point: {np.max(ei):.4f}")

In [ ]:
# Visualization
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Top: GP surrogate
ax1 = axes[0]
ax1.plot(X_test, objective(X_test), 'k--', label='True objective', linewidth=2)
ax1.plot(X_test, mu, 'b-', label='GP prediction', linewidth=2)
ax1.fill_between(X_test, mu - 2*std, mu + 2*std, alpha=0.3, color='blue')
ax1.scatter(X_train, y_train, c='red', s=100, zorder=5, label='Observations', edgecolors='black')
ax1.axhline(y=f_best, color='green', linestyle=':', label=f'Best so far = {f_best:.2f}')
ax1.set_ylabel('f(x)', fontsize=12)
ax1.set_title('Gaussian Process Surrogate Model', fontsize=14, fontweight='bold')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Bottom: Acquisition function
ax2 = axes[1]
ax2.fill_between(X_test, 0, ei, alpha=0.5, color='orange')
ax2.plot(X_test, ei, 'orange', linewidth=2)
next_x = X_test[np.argmax(ei)]
ax2.axvline(x=next_x, color='red', linestyle='--', linewidth=2, label=f'Next sample: x={next_x:.2f}')
ax2.set_xlabel('x', fontsize=12)
ax2.set_ylabel('Expected Improvement', fontsize=12)
ax2.set_title('Acquisition Function (Expected Improvement)', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎯 Bayesian Optimization balances:")
print("   - Exploitation: Sample where μ(x) is high")
print("   - Exploration: Sample where σ(x) is high (uncertain regions)")

---

# 📚 Conclusion

---

This notebook has taken you through the **deep mathematical foundations** that power modern machine learning:

1. **Linear Algebra**: Vectors, matrices, eigenvalues, SVD → PageRank, recommendations
2. **Calculus**: Gradients, chain rule, Jacobian → Backpropagation, Transformers
3. **Optimization**: Lagrange, convex, ADMM → SVMs, industrial-scale systems
4. **Probability**: Entropy, KL, Bayes → VAEs, generative models
5. **Advanced Methods**: Monte Carlo, flows → Sampling, density estimation
6. **Network Analysis**: Random walks → Link prediction
7. **Bayesian Optimization**: GPs, acquisition → Hyperparameter tuning

> **The Key Insight**: Mathematics is not just a prerequisite—it is the *language* in which ML discoveries are written. Mastering these foundations enables you to:
> - Debug models effectively
> - Design novel architectures
> - Understand cutting-edge research papers
> - Build production-grade ML systems

---

**Next Steps**: Apply these concepts to real datasets and implement more complex models!